# E9 - Random Forest - Boosting

Elaborado por GRUPO 1:

- Juanita Piraban Barbosa - 201216313
- Lorena Morales Rodríguez - 202027957
- Alejandro Barinas Guio - 201628859
- Jaime Humberto Trujillo Perea - 201920366
- Alexander Zapata Galindo - 201425426

## Car Price Prediction

Predict if the price of a car is low or high

In [53]:
%matplotlib inline
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [15]:
%matplotlib inline
import pandas as pd

data = pd.read_csv('Dataset/dataTraincarListings.csv')
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data['HighPrice'] = (data['Price'] > data['Price'].mean()).astype(int)
data = data.drop(['Model', 'Price'], axis=1)

data.head()

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,HighPrice
15,2016,29242,0,0,0,0,1,0,0,1
47,2015,26465,0,0,0,0,1,0,0,1
85,2012,46739,0,1,0,0,0,0,0,1
141,2017,41722,0,0,0,0,0,1,0,1
226,2014,77669,0,0,0,0,0,0,1,0


In [16]:
data.shape

(13150, 10)

In [17]:
y = data['HighPrice']
X = data.drop(['HighPrice'], axis=1)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Exercise 8.1

Estimate a Decision Tree Classifier Manually using the code created in the Notebook #13

Evaluate the accuracy on the testing set

# Exercise 8.2

Estimate a Bagging of 10 Decision Tree Classifiers Manually using the code created in the Notebook #7

Evaluate the accuracy on the testing set

# Exercise 8.3

Implement the variable max_features on the Decision Tree Classifier created in 11.1.

Compare the impact in the results by varing the parameter max_features

Evaluate the accuracy on the testing set

# Exercise 8.4

Estimate a Bagging of 10 Decision Tree Classifiers with `max_features = log(n_features)`

Evaluate the accuracy on the testing set

# Exercise 8.5

Using sklearn, train a RandomForestClassifier

Evaluate the accuracy on the testing set

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier()
clf

In [ ]:
#from sklearn import metrics
#clf.fit(X_train, y_train)
#y_pred = clf.predict(X_test)
#metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

In [ ]:
pd.Series(cross_val_score(clf, X_train, y_train, cv=10)).describe()

# Exercise 8.6

Find the best parameters of the RandomForestClassifier (max_depth, max_features, n_estimators)

Evaluate the accuracy on the testing set

## - n_estimators

In [ ]:
estimator_range = range(10, 310, 10)

accuracy_scores = []

# use 10-fold cross-validation with each value of n_estimators (WARNING: SLOW!)
for estimator in estimator_range:
    clf = RandomForestClassifier(n_estimators=estimator, random_state=1, n_jobs=-1)
    accuracy_scores.append(cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy').mean())

In [ ]:
plt.plot(estimator_range, accuracy_scores)
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')

## - max_features

In [ ]:
feature_range = range(1, len(feature_cols)+1)

accuracy_scores = []

# use 10-fold cross-validation with each value of max_features (WARNING: SLOW!)
for feature in feature_range:
    clf = RandomForestClassifier(n_estimators=200, max_features=feature, random_state=1, n_jobs=-1)
    accuracy_scores.append(cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy').mean())

## - Modelo de Random Forest ajustado a los mejores parámetros

In [ ]:
# max_features= XX & n_estimators= XX 
clf = RandomForestClassifier(n_estimators=200, max_features=6, random_state=1, n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
# compute feature importances
pd.DataFrame({'feature':feature_cols, 'importance':clf.feature_importances_}).sort_values('importance')

# Exercise 8.7 

Using xgboost train a XGBClassifier 

Evaluate the accuracy on the testing set

In [10]:
#!pip install xgboost

In [6]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)

In [42]:
from sklearn import metrics
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

[11:24:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.8989412897016361, 0.8790322580645161)

In [15]:
#Accuracy
acc= metrics.accuracy_score(y_pred, y_test.values)
print(acc)

0.8790322580645161


# Exercise 8.8

Using xgboost train a XGBClassifier 

Modify the parameters learning rate, gamma, colsample_bytree. Explain what each parameter means.

Evaluate the accuracy on the testing set

* Learning_rate: Se utiliza para evitar el over-fitting, Después de cada boosting,se reducen los pesos de las nuevas funciones para que el proceso sea más conservador.
* Gamma: Es la reducción de pérdida mínima requerida para hacer una partición adicional en un nodo hoja del árbol. Cuanto mayor sea gamma, más conservador será el algoritmo.
* Colsample_bytree: es la proporción de submuestra de columnas al construir cada árbol. El submuestreo ocurre una vez por cada árbol construido.

In [60]:
# Se establece la grilla de parámetros a calibrar
params = {
        'learning_rate': [0.08,0.09, 0.1, 0.11,0.12],
        'gamma': [0.5,0.7,0.9,1,1.1,1,2,1,3, 1.5, 2, 5],
        'colsample_bytree': [0.75, 0.8,0.85,0.9, 1.0],
        }

In [61]:
xgb = XGBClassifier(objective='binary:logistic',
                    silent=True, nthread=1)

In [62]:
# Se incluye función de timer para saber cuánto se esta demorando la ejecución
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [63]:
# Aquí se configuran los parámetros establecidos para la búsqueda de grilla, y se usa el auc como medida para determinar el rendimeinto de cada iteración

#las métricas folds, param_comb y n_jobs están bajitas para disminuir el tiempo de ejecución, para una mejor calibración se puede aumentar el número
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
param_comb = 7


folds=10
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=10, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )

#Timer y random search
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[11:34:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:34:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 0 minutes and 8.9 seconds.


In [64]:
#Impresión de resultados

print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
#results.to_csv('xgb-random-grid-search-results-01.csv', index=False)
print(results)


 All results:
{'mean_fit_time': array([1.28365331, 1.32155859, 1.12724202, 1.05133677, 1.18740382,
       1.00391269, 1.01555536]), 'std_fit_time': array([0.2461164 , 0.25237568, 0.22098507, 0.21371132, 0.21011206,
       0.20090669, 0.16844218]), 'mean_score_time': array([0.00647888, 0.00577562, 0.00542638, 0.01028583, 0.00851111,
       0.01020305, 0.00603106]), 'std_score_time': array([0.00067994, 0.00052254, 0.00042441, 0.00752354, 0.00504108,
       0.00725929, 0.00091901]), 'param_learning_rate': masked_array(data=[0.08, 0.09, 0.1, 0.12, 0.08, 0.11, 0.09],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[0.9, 5, 5, 0.7, 5, 1.5, 1],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_colsample_bytree': masked_array(data=[1.0, 1.0, 0.9, 0.85, 0.8, 0.75, 0.75],
             mask=[False, False, False, False,

Se ejecuta el modelo con los parámetros calibrados

In [65]:
xgb2 = XGBClassifier(learning_rate=0.12,gamma=0.7,colsamble_bytree=0.85)

In [66]:
xgb2.fit(X_train, y_train)
y_pred = xgb2.predict(X_test)
metrics.accuracy_score(y_pred, y_test.values)

[11:35:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "colsamble_bytree" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:35:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8820276497695853

### Conclusión
El accuracy del modelo calibrado es de 88.2% siendo mejor que el del modelo sin calibrar que cuenta con un accuracy de 87.9%